In [1]:
import pyspark

conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession(sc)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-90c6ecba-15c9-4c47-865b-1540d3435b75;1.0
	confs: [default]
	found io.delta#delta-core_2.12;0.8.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.2 in central
	found org.apache

In [2]:
!pip install koalas==1.8.1

     |████████████████████████████████| 1.4 MB 893 kB/s eta 0:00:01


In [3]:
import time
OBJECTURL_TEST = 's3a://test-container/playground/colors-test' + str(time.time()) + '.csv'
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

21/10/17 19:14:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
21/10/17 19:14:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+--------+-----+
|    name|color|
+--------+-----+
|   Luigi|Green|
|Princess| Pink|
|   Mario|  Red|
+--------+-----+



In [4]:
DELTA_URL='s3a://test-container/playground/delts-colors-test' + str(time.time())
gnames_df.write.format("delta").save(DELTA_URL)

In [5]:
data = spark.range(0, 5)
#data.write.format("delta").save("/tmp/delta-table")

In [11]:
data.write.format("delta").save("s3a://test-container/playground/to_overide")

AnalysisException: s3a://test-container/playground/to_overide already exists.

In [12]:
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/to_overide")

In [13]:
from databricks import koalas as ks
import pandas as pd
import numpy as np

In [14]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [15]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [16]:
kdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [17]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
kdf = ks.from_pandas(pdf)
print(type(kdf))
kdf

<class 'databricks.koalas.frame.DataFrame'>


,A,B,C,D
2013-01-01,-0.711055,0.225627,1.028010,0.861587
2013-01-02,-0.698692,-0.668969,0.513985,-0.506543
2013-01-03,0.770722,-1.601294,0.849015,-0.238955
2013-01-04,1.596659,-0.834587,-0.131964,-0.216534
2013-01-05,0.680701,-1.293159,0.585404,1.583542
2013-01-06,-0.689997,-1.680504,0.008660,0.245247


In [18]:
sdf = spark.createDataFrame(pdf)
sdf.show()

+-------------------+-------------------+--------------------+--------------------+
|                  A|                  B|                   C|                   D|
+-------------------+-------------------+--------------------+--------------------+
|-0.7110552325690844| 0.2256266465605143|   1.028009602206058|  0.8615871414898003|
|-0.6986918104874763|-0.6689693578564733|  0.5139848245778735| -0.5065427068760201|
| 0.7707221165142124|-1.6012937107688066|  0.8490154748686538|-0.23895543811339473|
| 1.5966585873695769|-0.8345871507597107|-0.13196437680788964|-0.21653363857988714|
| 0.6807005567050425| -1.293159005368727|  0.5854040626477828|  1.5835416305656298|
|-0.6899966854522469|-1.6805036398110058|0.008660405453540977| 0.24524722192265716|
+-------------------+-------------------+--------------------+--------------------+



In [19]:
kdf = sdf.to_koalas()

In [20]:
sdf.collect()

[Row(A=-0.7110552325690844, B=0.2256266465605143, C=1.028009602206058, D=0.8615871414898003),
 Row(A=-0.6986918104874763, B=-0.6689693578564733, C=0.5139848245778735, D=-0.5065427068760201),
 Row(A=0.7707221165142124, B=-1.6012937107688066, C=0.8490154748686538, D=-0.23895543811339473),
 Row(A=1.5966585873695769, B=-0.8345871507597107, C=-0.13196437680788964, D=-0.21653363857988714),
 Row(A=0.6807005567050425, B=-1.293159005368727, C=0.5854040626477828, D=1.5835416305656298),
 Row(A=-0.6899966854522469, B=-1.6805036398110058, C=0.008660405453540977, D=0.24524722192265716)]

In [21]:
kdf.to_parquet("s3a://test-container/playground/kdf")

In [22]:
ks.read_parquet("s3a://test-container/playground/kdf").head(10)

21/10/17 19:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,-0.711055,0.225627,1.028010,0.861587
1,-0.698692,-0.668969,0.513985,-0.506543
2,0.770722,-1.601294,0.849015,-0.238955
3,1.596659,-0.834587,-0.131964,-0.216534
4,0.680701,-1.293159,0.585404,1.583542
5,-0.689997,-1.680504,0.008660,0.245247


In [24]:
kdf.to_delta("s3a://test-container/playground/delta", mode=)

AttributeError: 'NoneType' object has no attribute 'mode'

In [21]:
ks.read_delta("s3a://test-container/playground/delta").head(10)

21/10/13 08:11:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,0.287319,1.309503,-0.601484,1.396977
1,0.613454,0.112073,1.067040,-1.363565
2,0.422918,0.160863,-0.159296,-0.201997
3,-1.007683,-0.022153,0.390490,0.615409
4,-0.627739,-2.077160,-0.956193,0.758404
5,-0.193660,1.829280,0.362110,-0.386733
